<a href="https://colab.research.google.com/github/delenarias/practica-mongo-neo4j/blob/main/scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creación de base e importación de ***Collection*** en la base. El script se ejectuta en consola.

In [ ]:
mongoimport --db inspections --collection grupal_city_inspections --type json --file act-grupal-city_inspections.json